<a href="https://colab.research.google.com/github/John-Akech/Formative-2---Data-Preprocessing_Final-Notebook/blob/master/Formative_2_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: Data Augmentation on CSV Files

Objective: Expand an existing dataset using synthetic data, perturbation, and augmentation techniques.

# Data Cleaning & Handling Missing Values

In [3]:
# Import the necessary libraries
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, KBinsDiscretizer
from imblearn.over_sampling import SMOTE
import uuid
import datetime

In [4]:
# Load the Dataset
try:
    # Load the dataset
    df = pd.read_csv('/content/customer_transactions.csv')
except FileNotFoundError:
    print("Error: The file 'customer_transactions.csv' was not found.")
    exit()

In [5]:
# Display basic information
print("\nDataset Overview:")
print(df.info())


Dataset Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   customer_id_legacy  150 non-null    int64  
 1   transaction_id      150 non-null    int64  
 2   purchase_amount     150 non-null    int64  
 3   purchase_date       150 non-null    object 
 4   product_category    150 non-null    object 
 5   customer_rating     140 non-null    float64
dtypes: float64(1), int64(3), object(2)
memory usage: 7.2+ KB
None


In [6]:
# Display first few rows
print("\nFirst 5 Rows:")
print(df.head())


First 5 Rows:
   customer_id_legacy  transaction_id  purchase_amount purchase_date  \
0                 151            1001              408    2024-01-01   
1                 192            1002              332    2024-01-02   
2                 114            1003              442    2024-01-03   
3                 171            1004              256    2024-01-04   
4                 160            1005               64    2024-01-05   

  product_category  customer_rating  
0           Sports              2.3  
1      Electronics              4.2  
2      Electronics              2.1  
3         Clothing              2.8  
4         Clothing              1.3  


In [7]:
# Ensure Correct Data Types
numerical_cols = ['purchase_amount', 'customer_rating']
categorical_cols = ['product_category']

In [8]:
# Convert purchase_date to datetime
df['purchase_date'] = pd.to_datetime(df['purchase_date'], errors='coerce')

In [9]:
# Extract year, month, and day from purchase_date
df['year'] = df['purchase_date'].dt.year
df['month'] = df['purchase_date'].dt.month
df['day'] = df['purchase_date'].dt.day

In [10]:
# Drop the original purchase_date column
df.drop(columns=['purchase_date'], inplace=True)

In [11]:
# Ensure numerical columns are numeric
for col in numerical_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')# Ensure categorical columns are category type
for col in categorical_cols:
    df[col] = df[col].astype('category')

In [12]:
# Verify updated data types
print("\nUpdated Data Types:")
print(df.dtypes)


Updated Data Types:
customer_id_legacy       int64
transaction_id           int64
purchase_amount          int64
product_category      category
customer_rating        float64
year                     int32
month                    int32
day                      int32
dtype: object


In [13]:
# Handle Missing Values
# Check for missing values
print("\nMissing Values Summary:")
print(df.isnull().sum())


Missing Values Summary:
customer_id_legacy     0
transaction_id         0
purchase_amount        0
product_category       0
customer_rating       10
year                   0
month                  0
day                    0
dtype: int64


In [14]:
# Impute numerical columns using median
imputer = SimpleImputer(strategy='median')
df[numerical_cols + ['year', 'month', 'day']] = imputer.fit_transform(df[numerical_cols + ['year', 'month', 'day']])

In [15]:
# Impute categorical columns using mode
for col in categorical_cols:
    mode_value = df[col].mode()[0]
    df[col].fillna(mode_value, inplace=True)
    print(f"\nFilled missing values in {col} with mode: {mode_value}")


Filled missing values in product_category with mode: Sports


<ipython-input-15-b5588db212dd>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mode_value, inplace=True)


In [16]:
# Predictive modeling for remaining missing values in customer_rating
if df['customer_rating'].isnull().any():
    X_train = df[df['customer_rating'].notnull()][['purchase_amount']]
    y_train = df[df['customer_rating'].notnull()]['customer_rating']
    X_missing = df[df['customer_rating'].isnull()][['purchase_amount']]

    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    df.loc[df['customer_rating'].isnull(), 'customer_rating'] = regressor.predict(X_missing)

In [17]:
# Verify missing values are handled
print("\nMissing Values After Imputation:")
print(df.isnull().sum())


Missing Values After Imputation:
customer_id_legacy    0
transaction_id        0
purchase_amount       0
product_category      0
customer_rating       0
year                  0
month                 0
day                   0
dtype: int64


# Data Augmentation Strategies

In [18]:
# Apply Random Noise to Numerical Columns
# Add random noise to purchase_amount
noise_factor = 0.05  # Adjust noise factor for better augmentation
original_std = df['purchase_amount'].std()
df['purchase_amount'] += np.random.normal(0, noise_factor * original_std, df.shape[0])

print("\nRandom Noise Applied to purchase_amount:")
print(df[['purchase_amount']].head())


Random Noise Applied to purchase_amount:
   purchase_amount
0       396.610631
1       331.379468
2       425.465727
3       257.868081
4        55.457181


In [19]:
# Transform Skewed Features
# Check skewness
skewness = df['purchase_amount'].skew()
print(f"\nSkewness of purchase_amount: {skewness}")

# Apply log transformation if skewed
if skewness > 1:
    df['purchase_amount'] = np.log1p(df['purchase_amount'])
    print("\nLog Transformation Applied to purchase_amount.")

print("\nTransformed purchase_amount:")
print(df[['purchase_amount']].head())


Skewness of purchase_amount: 0.05547951416809389

Transformed purchase_amount:
   purchase_amount
0       396.610631
1       331.379468
2       425.465727
3       257.868081
4        55.457181


In [20]:
# Generate Synthetic Data (Choose One Approach)

# Approach 1: Discretize Target Variable and Use SMOTE
def augment_with_smote(df):
    # Encode categorical variables
    X = df.drop(columns=['customer_id_legacy', 'transaction_id', 'customer_rating'])
    y = df['customer_rating']

    # One-hot encode categorical features
    encoder = OneHotEncoder(drop='first', sparse_output=False)
    X_encoded = encoder.fit_transform(X[categorical_cols])
    X_encoded = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(categorical_cols))
    X_final = pd.concat([X.drop(columns=categorical_cols), X_encoded], axis=1)

    # Discretize the continuous target into bins
    discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
    y_discrete = discretizer.fit_transform(y.values.reshape(-1, 1)).ravel()

    # Apply SMOTE to the discretized target
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_final, y_discrete)

    # Decode the discretized target back to the original scale
    y_resampled_continuous = discretizer.inverse_transform(y_resampled.reshape(-1, 1)).ravel()

    # Combine resampled data into a DataFrame
    synthetic_data = pd.DataFrame(X_resampled, columns=X_final.columns)
    synthetic_data['customer_rating'] = y_resampled_continuous

    # Decode one-hot-encoded features back to original format
    def decode_one_hot(encoded_df, original_df):
        decoded_df = pd.DataFrame(index=encoded_df.index)
        for col in categorical_cols:
            one_hot_cols = [c for c in encoded_df.columns if c.startswith(col)]
            decoded_df[col] = encoded_df[one_hot_cols].idxmax(axis=1).str.replace(f"{col}_", "")
        return decoded_df

    decoded_categoricals = decode_one_hot(synthetic_data, X)
    synthetic_data = pd.concat([synthetic_data.drop(columns=[c for c in synthetic_data.columns if any(cat in c for cat in categorical_cols)]), decoded_categoricals], axis=1)

    # Generate synthetic IDs
    synthetic_data['customer_id_legacy'] = [uuid.uuid4().int % 10**9 for _ in range(synthetic_data.shape[0])]
    synthetic_data['transaction_id'] = range(df['transaction_id'].max() + 1, df['transaction_id'].max() + 1 + synthetic_data.shape[0])

    return synthetic_data

In [21]:
# Approach 2: Interpolation for Continuous Target
def augment_with_interpolation(df):
    # Function to generate synthetic samples via interpolation
    def interpolate_data(X, y, n_samples):
        synthetic_X = []
        synthetic_y = []

        for _ in range(n_samples):
            # Randomly select two samples
            idx1, idx2 = np.random.choice(len(X), size=2, replace=False)
            alpha = np.random.uniform(0, 1)  # Interpolation factor

            # Interpolate features and target
            synthetic_X.append(alpha * X.iloc[idx1] + (1 - alpha) * X.iloc[idx2])
            synthetic_y.append(alpha * y.iloc[idx1] + (1 - alpha) * y.iloc[idx2])

        synthetic_X = pd.DataFrame(synthetic_X, columns=X.columns)
        synthetic_y = pd.Series(synthetic_y, name=y.name)
        return synthetic_X, synthetic_y

    # Prepare data for augmentation
    X = df.drop(columns=['customer_id_legacy', 'transaction_id', 'customer_rating'])
    y = df['customer_rating']

    # Generate synthetic data
    n_synthetic_samples = len(df)  # Generate as many synthetic samples as the original dataset
    synthetic_X, synthetic_y = interpolate_data(X, y, n_synthetic_samples)

    # Combine synthetic data into a DataFrame
    synthetic_data = pd.concat([synthetic_X, synthetic_y], axis=1)
    synthetic_data.columns = X.columns.tolist() + ['customer_rating']

    # Generate synthetic IDs
    synthetic_data['customer_id_legacy'] = [uuid.uuid4().int % 10**9 for _ in range(synthetic_data.shape[0])]
    synthetic_data['transaction_id'] = range(df['transaction_id'].max() + 1, df['transaction_id'].max() + 1 + synthetic_data.shape[0])

    return synthetic_data

In [22]:
# Choose an augmentation approach
augmentation_method = "smote"  # Change to "interpolation" if needed

if augmentation_method == "smote":
    synthetic_data = augment_with_smote(df)
elif augmentation_method == "interpolation":
    synthetic_data = augment_with_interpolation(df)

# Concatenate synthetic data with the original dataset
df_augmented = pd.concat([df, synthetic_data], axis=0).reset_index(drop=True)

print("\nSynthetic Data Generated:")
print(synthetic_data.head())


Synthetic Data Generated:
   purchase_amount    year  month  day  customer_rating product_category  \
0       396.610631  2024.0    1.0  1.0         1.666667           Sports   
1       331.379468  2024.0    1.0  2.0         4.333333      Electronics   
2       425.465727  2024.0    1.0  3.0         1.666667      Electronics   
3       257.868081  2024.0    1.0  4.0         3.000000         Clothing   
4        55.457181  2024.0    1.0  5.0         1.666667         Clothing   

   customer_id_legacy  transaction_id  
0           974918933            1151  
1           131039049            1152  
2           376203150            1153  
3           561642990            1154  
4           436704860            1155  


# Export the Augmented Data

In [23]:
# Export the Augmented Dataset

# Save the final dataset
output_file = 'customer_transactions_augmented.csv.'
df_augmented.to_csv(output_file, index=False)

print(f"\nFinal Preprocessed Dataset Saved Successfully as '{output_file}'!")


Augmented Dataset Saved Successfully as 'customer_transactions_augmented_20250316_190135.csv'!


 # Part 2: Merging Datasets with Transitive Properties

 Objective: Merge two different datasets with shared but indirect relationships between entities.

In [24]:
# Import the necessary libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from datetime import datetime

In [25]:
# Load the Datasets
try:
    # Load the augmented transactions dataset
    transactions_df = pd.read_csv('/content/customer_transactions_augmented.csv.csv')
except FileNotFoundError:
    print("Error: The file 'customer_transactions_augmented.csv' was not found.")
    exit()

Error: The file 'customer_transactions_augmented.csv' was not found.


In [26]:
try:
    # Load the social profiles dataset
    social_profiles_df = pd.read_csv('/content/customer_social_profiles.csv')
except FileNotFoundError:
    print("Error: The file 'customer_social_profiles.csv' was not found.")
    exit()

In [27]:
try:
    # Load the ID mapping dataset
    id_mapping_df = pd.read_csv('/content/id_mapping.csv')
except FileNotFoundError:
    print("Error: The file 'id_mapping.csv' was not found.")
    exit()

In [27]:
# Display basic information about all datasets
print("\nTransactions Dataset Overview:")
print(transactions_df.info())
print("\nSocial Profiles Dataset Overview:")
print(social_profiles_df.info())
print("\nID Mapping Dataset Overview:")
print(id_mapping_df.info())

In [ ]:
# Map IDs Using id_mapping.csv
# Merge transactions_df with id_mapping_df to get customer_id_new
transactions_mapped = pd.merge(
    transactions_df,
    id_mapping_df,
    left_on='customer_id_legacy',
    right_on='customer_id_legacy',
    how='left'
)
print("\nTransactions Mapped with New IDs:")
print(transactions_mapped.head())

In [ ]:
# Debugging: Check if customer_id_new exists
if 'customer_id_new' not in transactions_mapped.columns:
    raise KeyError("Column 'customer_id_new' is missing after merging transactions_df with id_mapping_df.")

# Merge social_profiles_df with id_mapping_df to get customer_id_legacy
social_profiles_mapped = pd.merge(
    social_profiles_df,
    id_mapping_df,
    left_on='customer_id_new',
    right_on='customer_id_new',
    how='left'
)
print("\nSocial Profiles Mapped with Legacy IDs:")
print(social_profiles_mapped.head())

In [ ]:
# Debugging: Check if customer_id_legacy exists
if 'customer_id_legacy' not in social_profiles_mapped.columns:
    raise KeyError("Column 'customer_id_legacy' is missing after merging social_profiles_df with id_mapping_df.")

In [ ]:
# Merge Both Datasets Based on Transitive Relationships
# Merge the two mapped datasets on customer_id_new
merged_df = pd.merge(
    transactions_mapped,
    social_profiles_mapped,
    on='customer_id_new',
    how='outer'
)
print("\nFinal Merged Dataset:")
print(merged_df.head())

In [ ]:
# Debugging: Check for missing columns
missing_columns = ['customer_id_new', 'customer_id_legacy', 'engagement_score']
for col in missing_columns:
    if col not in merged_df.columns:
        print(f"Warning: Column '{col}' is missing in the final merged dataset.")

In [ ]:
# Handle conflicts where one customer ID maps to multiple entries
# Aggregate duplicate rows by taking the mean of numerical columns and mode of categorical columns
def aggregate_duplicates(df):
    # Group by customer_id_new and aggregate
    aggregation_dict = {
        'purchase_amount': 'mean',
        'customer_rating': 'mean',
        'engagement_score': lambda x: x.sum() if 'engagement_score' in df.columns else np.nan,
        'purchase_interest_score': 'mean'
    }

    # Only include customer_id_legacy if it exists
    if 'customer_id_legacy' in df.columns:
        aggregation_dict['customer_id_legacy'] = lambda x: x.mode()[0] if not x.empty else np.nan

    aggregated_df = df.groupby('customer_id_new', as_index=False).agg(aggregation_dict)
    return aggregated_df

merged_df = aggregate_duplicates(merged_df)
print("\nAggregated Merged Dataset:")
print(merged_df.head())

In [ ]:
# Create a Customer Engagement Score
# Combine transaction history (purchase_amount) and engagement_score
if 'engagement_score' in merged_df.columns:
    merged_df['customer_engagement_score'] = (
        merged_df['purchase_amount'] * 0.6 +
        merged_df['engagement_score'] * 0.4
    )
else:
    print("Warning: 'engagement_score' column is missing. Skipping customer engagement score calculation.")
    merged_df['customer_engagement_score'] = merged_df['purchase_amount'] * 0.6

print("\nCustomer Engagement Score Added:")
print(merged_df[['customer_id_new', 'customer_engagement_score']].head())

In [ ]:
# Engineer predictive behavioral features
# Moving average of transactions (rolling window of 3)
merged_df.sort_values(by=['customer_id_new'], inplace=True)
merged_df['moving_avg_purchase'] = merged_df.groupby('customer_id_new')['purchase_amount'].transform(
    lambda x: x.rolling(window=3, min_periods=1).mean()
)
print("\nMoving Average of Purchases Added:")
print(merged_df[['customer_id_new', 'purchase_amount', 'moving_avg_purchase']].head())

In [ ]:
# Time-based aggregation of purchases (e.g., monthly total)
# Convert purchase_date to datetime
if 'purchase_date' in merged_df.columns:
    merged_df['month'] = merged_df['purchase_date'].dt.to_period('M').astype(str)
    monthly_aggregation = merged_df.groupby(['customer_id_new', 'month'])['purchase_amount'].sum().reset_index()
    monthly_aggregation.rename(columns={'purchase_amount': 'monthly_total_purchase'}, inplace=True)
    merged_df = pd.merge(merged_df, monthly_aggregation, on=['customer_id_new', 'month'], how='left')
    print("\nMonthly Aggregation of Purchases Added:")
    print(merged_df[['customer_id_new', 'month', 'monthly_total_purchase']].head())
else:
    print("Warning: 'purchase_date' column is missing. Skipping time-based aggregation.")

# TF-IDF on review_sentiment
if 'review_sentiment' in merged_df.columns:
    tfidf = TfidfVectorizer(max_features=10)  # Limit to top 10 features for simplicity
    tfidf_matrix = tfidf.fit_transform(merged_df['review_sentiment'].fillna(''))
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())
    merged_df = pd.concat([merged_df.reset_index(drop=True), tfidf_df], axis=1)
    print("\nTF-IDF Features Added:")
    print(merged_df.head())
else:
    print("Warning: 'review_sentiment' column is missing. Skipping TF-IDF feature extraction.")

In [ ]:
# Export the Final Preprocessed Data
# Save the final dataset
output_file = 'final_customer_data_group17.csv'
merged_df.to_csv(output_file, index=False)

print(f"\nFinal Preprocessed Dataset Saved Successfully as '{output_file}'!")

# Part 3: Data Consistency and Quality Checks
Objective: Ensure that the preprocessed dataset is clean, structured, and machine learning-ready.